## CTAB-GAN generated dataset
- loan_status postprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch 
import os
import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print(device)

cuda


In [3]:
## Load original data
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/CTAB-GAN/Real_Datasets/train_category.csv"
data = pd.read_csv(data_path, low_memory=False)

##load test data
test_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/CTAB-GAN/Real_Datasets/test_category.csv"
test_data = pd.read_csv(test_data_path)

## Load original data
fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/CTAB-GAN/Fake_Datasets/ctab-gan.csv"
fake = pd.read_csv(fake_path, low_memory=False)

In [4]:
for col in fake.columns:
    print(f"{col} features \n: {fake[col].unique()}\n")

last_fico_range_high features 
: [563.86206055 534.6661377  533.87994385 ... 540.29870605 526.31994629
 553.51928711]

annual_inc features 
: [139317.95337149  51065.53110064 110901.12064818 ...  40028.00715938
 105753.94375303 184296.58678978]

dti features 
: [22.74494553 10.4570303  24.68014145 ... 15.84842777 12.93407726
  8.72013092]

mo_sin_old_rev_tl_op features 
: [346.71786499 137.81939697 121.98181152 ... 185.96505737 133.13142395
 296.23239136]

revol_util features 
: [0.80004752 0.68297833 0.09741139 ... 0.57721931 0.3389667  0.88475329]

int_rate features 
: [0.17203716 0.1054262  0.17335222 ... 0.09017627 0.17156107 0.15861231]

installment features 
: [677.74697076 152.29754169 585.05818331 ... 759.53978809 510.73694392
 733.89783621]

avg_cur_bal features 
: [  792.5735191   4002.25032131 23298.90734261 ...   749.39040229
 16972.06465209  2337.85241843]

revol_bal features 
: [ 8090.16372122 13167.17832908 17975.91762104 ...  6671.30368835
 16942.00257425 13812.65133719

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116458 entries, 0 to 1116457
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   last_fico_range_high  1116458 non-null  float64
 1   annual_inc            1116458 non-null  float64
 2   dti                   1116458 non-null  float64
 3   mo_sin_old_rev_tl_op  1116458 non-null  float64
 4   revol_util            1116458 non-null  float64
 5   int_rate              1116458 non-null  float64
 6   installment           1116458 non-null  float64
 7   avg_cur_bal           1116458 non-null  float64
 8   revol_bal             1116458 non-null  float64
 9   debt_settlement_flag  1116458 non-null  object 
 10  sub_grade             1116458 non-null  object 
 11  home_ownership        1116458 non-null  object 
 12  purpose               1116458 non-null  object 
 13  grade                 1116458 non-null  object 
 14  total_pymnt           1116458 non-

In [6]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540000 entries, 0 to 539999
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   last_fico_range_high  540000 non-null  float64
 1   annual_inc            540000 non-null  float64
 2   dti                   540000 non-null  float64
 3   mo_sin_old_rev_tl_op  540000 non-null  float64
 4   revol_util            540000 non-null  float64
 5   int_rate              540000 non-null  float64
 6   installment           540000 non-null  float64
 7   avg_cur_bal           540000 non-null  float64
 8   revol_bal             540000 non-null  float64
 9   debt_settlement_flag  540000 non-null  object 
 10  sub_grade             540000 non-null  object 
 11  home_ownership        540000 non-null  object 
 12  purpose               540000 non-null  object 
 13  grade                 540000 non-null  object 
 14  total_pymnt           540000 non-null  float64
 15  

In [7]:
fake['loan_status'].value_counts()

loan_status
1    540000
Name: count, dtype: int64